# ODOT - Exploratory Analysis Notebook

<div style="color:red; border:1px solid; padding:5px; max-width:800px; font-size:80%;">
The purpose of this test and the following analysis was to evaluate ODOT’s new Activity Based Model (ABM); 
specifically the ability of the ABM to provide information about emerging technologies.  To help to achieve 
that purpose a realistic, but fictitious, set of regional ABM inputs was developed.  At the end of this 
beta test, several flaws in the performance measure creation and methodology were noted as potential 
improvements for future analysis, but were not corrected in this dataset and resulting analysis.  The 
information in this data and analysis serves as an example for how to use TMIP-EMAT using realistic data.  
This dataset and analysis should not be used to draw any specific conclusions about transportation policy’s 
impact on system performance and outcomes.
</div>


In this notebook, we walk through an Exploratory Analysis for the 
ODOT SOABM using TMIP-EMAT.  The documentation here presumes that the
model scoping, initial experimental designs, and initial core model runs,
and meta-model development have all been completed previously.

In [ ]:
import os
import numpy
import pandas

In [ ]:
import emat.versions()

Enable some logging. This is optional but convenient, especially for
keeping track of run-time.

In [ ]:
from emat.util.loggers import log_to_stderr, TimingLog
log = log_to_stderr(level=25)
log.log(25, 'Logging Starts')

## Load Existing Scope, Data, and MetaModel

Connect to DB file with populated LHS experiment design, and read scope 
and experimental data.

In [ ]:
db = emat.SQLiteDB("soabm_analysis_v2.db", initialize=False)

In [ ]:
db.read_scope_names()

In [ ]:
scope = db.read_scope('SOABM')
scope.info()

In [ ]:
db.read_design_names('SOABM')

Load the data from the core model experiments into a `pandas.DataFrame`.

In [ ]:
core_experiments = db.read_experiment_all(scope_name='SOABM', design_name='odot_lhs', ensure_dtypes=True)

In [ ]:
mm = db.read_metamodel('SOABM', 1)

In [ ]:
meta_experiments = db.read_experiment_all(scope_name='SOABM', design_name='mc', ensure_dtypes=True, source=1)
meta_experiments.info()

# Directed Search 

## Policy Optimization: Search Over Levers

Note: the use of the `cache_file` is argument, and allows re-loading the 
result of an optimization from a file instead of re-running the entire
optimization process.

In [ ]:
result = mm.optimize(
    nfe=5000, 
    searchover='levers', 
    cache_file='./optimization_cache/odot_demo_search_over_levers_1.gz',
)

In [ ]:
pc = result.par_coords()
pc

After viewing the results above, suppose we have decided that we're not actually
interested in optimizing for auto ownership or non-mandatory tour making.  We 
can set these measures to `INFO` (0) in the scope to not optimize them, and re-run.

In [ ]:
for i in ['Number of Autos Owned Per Household', 'Percent of Non-Mandatory Tours']:
    mm.scope[i].kind = 0

In [ ]:
result1 = mm.optimize(
    nfe=5000, 
    searchover='levers', 
    cache_file='./optimization_cache/odot_demo_search_over_levers_2.gz',
)

In [ ]:
pc1 = result1.par_coords()
pc1

## Worst Case Planning: Search Over Uncertainties

In [ ]:
worst = mm.optimize(
    nfe=2500, 
    searchover='uncertainties', 
    reverse_targets = True,
    cache_file='./optimization_cache/odot_demo_search_over_uncs_1.gz',
)

In [ ]:
wc = worst.par_coords()
wc

## Robust Search

In [ ]:
from emat import Measure
import functools

robustness_functions = [
    Measure(
        'Worst Case Accessibility',
        kind=Measure.MAXIMIZE,
        variable_name='Percentage of Population with Access to 50k Jobs by Car within 20mins in PM',
        function=numpy.min,
    ),

    Measure(
        'Worst Case Auto VHT',
        kind=Measure.MINIMIZE,
        variable_name='Thousands of Auto Hours Traveled',
        function=numpy.max,
    ),

    Measure(
        'Worst Case Congested Interstate Miles',
        kind=Measure.MINIMIZE,
        variable_name='Percent of Interstate Miles over 90% V/C Ratio During the PM Peak',
        function=max,
    ),

]

In [ ]:
from emat.util.distributed import get_client
client = get_client(n_workers=6, threads_per_worker=1)

In [ ]:
robust = mm.robust_optimize(
        robustness_functions,
        scenarios=200,
        nfe=2000,
        evaluator=client,
        convergence_freq=100,
        cache_file='./robust_cache/odot_demo_5.gz',
        check_extremes=2,
)

In [ ]:
robust.par_coords()

In [ ]:
worst_1 = mm.optimize(
    nfe=2500, 
    searchover='uncertainties', 
    reverse_targets = True,
    reference = ema_workbench.Policy(**{
         'Transit Everywhere': True,
         'Transit LOS': -10,
         'Parking Rates': 20,
         'Active Transport Speed': 2,
    })
)

In [ ]:
worst_1.par_coords()